# Preuzimanje podatkovnog seta dogs-vs-cats

 Preuzimanje s Kaggle servisa (korištenjem Kaggle API-ja)

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"erikuranec","key":"c537f5a7326999bd4eb8b62a65a7a6f4"}'}

In [ ]:
#!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

 99% 808M/812M [00:03<00:00, 295MB/s]
100% 812M/812M [00:03<00:00, 253MB/s]


Preuzeti podatkovni set je potrebno raspakirati i zatim se dobivanju 3 datoteke. Podatkovni set je već podijeljen na trening i testni set.

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

Potrebno je raspakirati train.zip datoteku.

In [ ]:
!unzip -qq train.zip

## Uzorkovanje cjelovitog podatkovnog seta

S obzirom da je cilj realizirati scenarij ograničenih podataka u sljedećim linijama će se početni podatkovni set ograničiti te uvesti neravnoteža u klase.
Podjela će biti sljedeća:


*   test: 1400 fotografija pasa, 600 fotografija mačaka
*   train: 1400 fotografija pasa, 600 fotografija mačaka
*   validation: 700 fotografija pasa, 300 fotografija mačaka

manje zastupljena klasa je mačka.





In [ ]:
import os
import shutil
import pathlib

original_dir = pathlib.Path("train")  # Početni direktorij gdje su originalne instance spremljene
new_base_dir = pathlib.Path("/content/cats_vs_dogs_baseline_dataset")  # Novi bazni direktorij za uzorkovani podset

def make_subset(subset_name, start_index, end_index, dog_percent=0.7):
    total_images = end_index - start_index
    dog_images = int(total_images * dog_percent)
    cat_images = total_images - dog_images

    start_index_cat = start_index
    end_index_cat = start_index + cat_images

    start_index_dog = start_index
    end_index_dog = start_index + dog_images

    for category, start_index, end_index in [("cat", start_index_cat, end_index_cat),
                                             ("dog", start_index_dog, end_index_dog)]:
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]

        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Kreiranje podsetova za trening, test i validaciju s neuravnoteženim klasama (dominacija klase "pas")
make_subset("train", start_index=0, end_index=2000, dog_percent=0.7)
make_subset("validation", start_index=2000, end_index=3000,dog_percent=0.7)
make_subset("test", start_index=3000, end_index=5000,dog_percent=0.7)


## Kompresija podatkovnog seta

In [ ]:
# Samo zip datoteke mogu biti preuzete iz Colaba
!zip -r cats_vs_dogs_baseline_dataset.zip cats_vs_dogs_baseline_dataset